# Homework5 Code

**10185501402 孙秋实 2018级**

给定盒子和球组成的隐马尔可夫模型 $\lambda=(\mathrm{A}, \mathrm{B}, \pi)$, 其中,
$$
\mathrm{A}=\left[\begin{array}{ccc}
0.5 & 0.1 & 0.4 \\
0.3 & 0.5 & 0.2 \\
0.2 & 0.2 & 0.6
\end{array}\right], \quad \mathrm{B}=\left[\begin{array}{cc}
0.5 & 0.5 \\
0.4 & 0.6 \\
0.7 & 0.3
\end{array}\right], \quad \pi=(0.2,0.3,0.5)^{\mathrm{T}}
$$
设T $=8, \mathrm{O}=(red, white, red, red, white, red, white, white)$, 试用前向后向概率计算P $\left(\mathrm{i}_{4}=\mathrm{q}_{3} \mid \mathrm{O}, \lambda\right)$

编程验证Homework5计算的正确性

In [10]:
import numpy as np

In [11]:
class HiddenMarkov: # 定义隐马尔可夫模型
    def __init__(self):
        self.alphas = None
        self.forward_P = None
        self.betas = None
        self.backward_P = None
    
    # 前向算法
    def forward(self, Q, V, A, B, O, PI):
        N = len(Q)  # 状态序列的大小
        M = len(O)  # 观测序列的大小
        alphas = np.zeros((N, M)) # 初始化前向概率
        T = M  # 时刻数=观测序列数
        # 遍历每一个时刻，计算前向概率alpha值
        for t in range(T):  
            indexOfO = V.index(O[t]) # 得到序列对应的索引
            for i in range(N):# 遍历状态序列
                # 初始化alpha初值
                if t == 0: 
                    # P176 公式(10.15)
                    alphas[i][t] = PI[t][i] * B[i][indexOfO]  
                    print('alpha1(%d) = p%db%db(o1) = %f' % (i+1, i, i, alphas[i][t]))
                else:
                    # P176 公式(10.16)
                    alphas[i][t] = np.dot([alpha[t - 1] for alpha in alphas], [a[i] for a in A]) * B[i][indexOfO]  
                    print('alpha%d(%d) = [sigma alpha%d(i)ai%d]b%d(o%d) = %f' % (t+1, i+1, t - 1, i, i, t, alphas[i][t]))
        self.forward_P = np.sum([alpha[M - 1] for alpha in alphas]) 
        self.alphas = alphas        
        
    # 后向算法
    def backward(self, Q, V, A, B, O, PI):  
        N = len(Q)  # 状态序列的大小
        M = len(O) # 观测序列的大小
        # 初始化后向概率beta值
        betas = np.ones((N, M))  
        for i in range(N):
            print('beta%d(%d) = 1' % (M, i+1))
        # 对观测序列逆向遍历
        for t in range(M - 2, -1, -1):
            indexOfO = V.index(O[t + 1])  # 得到序列对应的索引
            for i in range(N): # 遍历状态序列
                betas[i][t] = np.dot(np.multiply(A[i], [b[indexOfO] for b in B]), [beta[t + 1] for beta in betas])
                realT = t + 1
                realI = i + 1
                print('beta%d(%d) = sigma[a%djbj(o%d)beta%d(j)] = (' % (realT, realI, realI, realT + 1, realT + 1),end='')
                for j in range(N):
                    print("%.2f * %.2f * %.2f + " % (A[i][j], B[j][indexOfO], betas[j][t + 1]), end='')
                print("0) = %.3f" % betas[i][t])
        indexOfO = V.index(O[0]) # 取出第一个值
        self.betas = betas
        P = np.dot(np.multiply(PI, [b[indexOfO] for b in B]), [beta[0] for beta in betas])
        self.backward_P = P
        print("P(O|lambda) = ", end="")
        for i in range(N):
            print("%.1f * %.1f * %.5f + " % (PI[0][i], B[i][indexOfO], betas[i][0]), end="")
        print("0 = %f" % P)

In [12]:
Q = [1, 2, 3]
V = ['r', 'w']
A = [[0.5, 0.1, 0.4], [0.3, 0.5, 0.2], [0.2, 0.2, 0.6]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['r', 'w', 'r', 'r', 'w', 'r', 'w', 'w']
PI = [[0.2, 0.3, 0.5]]

HMM = HiddenMarkov()
print("---calculating alpha---")
HMM.forward(Q, V, A, B, O, PI)
print("---calculating beta---")
HMM.backward(Q, V, A, B, O, PI)

---calculating alpha---
alpha1(1) = p0b0b(o1) = 0.100000
alpha1(2) = p1b1b(o1) = 0.120000
alpha1(3) = p2b2b(o1) = 0.350000
alpha2(1) = [sigma alpha0(i)ai0]b0(o1) = 0.078000
alpha2(2) = [sigma alpha0(i)ai1]b1(o1) = 0.084000
alpha2(3) = [sigma alpha0(i)ai2]b2(o1) = 0.082200
alpha3(1) = [sigma alpha1(i)ai0]b0(o2) = 0.040320
alpha3(2) = [sigma alpha1(i)ai1]b1(o2) = 0.026496
alpha3(3) = [sigma alpha1(i)ai2]b2(o2) = 0.068124
alpha4(1) = [sigma alpha2(i)ai0]b0(o3) = 0.020867
alpha4(2) = [sigma alpha2(i)ai1]b1(o3) = 0.012362
alpha4(3) = [sigma alpha2(i)ai2]b2(o3) = 0.043611
alpha5(1) = [sigma alpha3(i)ai0]b0(o4) = 0.011432
alpha5(2) = [sigma alpha3(i)ai1]b1(o4) = 0.010194
alpha5(3) = [sigma alpha3(i)ai2]b2(o4) = 0.011096
alpha6(1) = [sigma alpha4(i)ai0]b0(o5) = 0.005497
alpha6(2) = [sigma alpha4(i)ai1]b1(o5) = 0.003384
alpha6(3) = [sigma alpha4(i)ai2]b2(o5) = 0.009288
alpha7(1) = [sigma alpha5(i)ai0]b0(o6) = 0.002811
alpha7(2) = [sigma alpha5(i)ai1]b1(o6) = 0.002460
alpha7(3) = [sigma alpha5(i

In [13]:
# 计算答案
print("alpha4(3)=", HMM.alphas[3-1][4-1])
print("beta4(3)=", HMM.betas[3-1][4-1])
print("P(O|lambda)=", HMM.backward_P[0])
result = (HMM.alphas[3-1][4-1] * HMM.betas[3-1][4-1]) / HMM.backward_P[0]
print("P(i4=q3|O,lambda) =",result)

alpha4(3)= 0.043611119999999996
beta4(3)= 0.04280618
P(O|lambda)= 0.0034767094492824
P(i4=q3|O,lambda) = 0.5369518160647322


与手推计算结果一致

**End of Homework5 Code**